## Stability of caloric sufficiency w.r.t. diet changes
#### What we are trying to do is find what the % of meat consumption in diet needs to be for caloric sufficiency to remain the same from 2000 to 2050.
To achieve that we will write a simple function that calculates the ∆cal_suff that only depends on diet and then try to find its root(s).

In [11]:
import pandas as pd
import numpy as np
from scipy.optimize import root

In [19]:
def f(x):
    '''example multivariate vectorial function'''
    return [np.log(x[0]**2) - x[1],
           x[1]**3 - 2 * x[0]]

root(f, x0=[-2, 1], method='hybr')

    fjac: array([[-0.91524006, -0.40290897],
       [ 0.40290897, -0.91524006]])
     fun: array([ 3.38906680e-12, -5.75206549e-12])
 message: 'The solution converged.'
    nfev: 17
     qtf: array([ 5.21627853e-10, -4.52168491e-09])
       r: array([ 3.96506486, -0.44605862, -3.36515374])
  status: 1
 success: True
       x: array([-0.58963483, -1.05650374])

In [16]:
all_countries = [
    
    'Argentina', 'Australia', 'Bulgaria',
    'Belize', 'Brazil', 'Canada', 'Denmark',
    'France', 'Hungary', 'Kazakhstan',
    'Lithuania', 'Latvia', 'Paraguay',
    'Ukraine', 'Uruguay', 'United States of America',
    'Belarus', 'Botswana', 'Estonia',
    'Finland', 'Georgia', 'Iran', 'Libya',
    'Lesotho', 'Moldova', 'Mongolia',
    'Norway', 'New Zealand', 'Poland',
    'Portugal', 'Russia', 'Swaziland',
    'Tunisia',
    'United Arab Emirates', 'Belgium',
    'Brunei', 'Cyprus', 'Djibouti', 'Algeria',
    'Gambia', 'Iraq', 'Israel', 'Jordan',
    'Japan', 'South Korea', 'Kuwait',
    'Lebanon', 'Montenegro',
    'Mauritania', 'Netherlands', 'Saudi Arabia', 'Singapore', 'Trinidad and Tobago', 'Yemen',
    'Afghanistan', 'Angola', 'Burundi',
    'Bangladesh', 'Cameroon',
    'Democratic Republic of the Congo',
    'Republic of Congo', 'Dominican Republic', 'Egypt', 'Eritrea', 'Ethiopia',
    'Ghana', 'Guinea Bissau',
    'Guatemala', 'Haiti', 'India', 'Jamaica',
    'Kenya', 'Sri Lanka', 'Madagascar',
    'Mozambique', 'Malawi', 'Niger',
    'Nigeria', 'Nepal', 'Oman', 'Pakistan',
    'Philippines', 'Puerto Rico', 'North Korea', 'Rwanda', 'El Salvador',
    'Syria', 'Togo', 'Uganda',
    'Benin', 'Burkina Faso', 'Bolivia',
    'China', 'Ivory Coast', 'Colombia',
    'Costa Rica', 'United Kingdom',
    'Guinea', 'Honduras', 'Indonesia',
    'Ireland', 'Luxembourg', 'Mexico',
    'Malaysia', 'Panama', 'Sudan',
    'Senegal', 'Sierra Leone', 'Tajikistan',
    'Tanzania', 'Uzbekistan', 'Vietnam',
    'Zambia', 'Gabon', 'Equatorial Guinea',
    'Morocco', 'Peru',
    'Albania', 'Austria', 'Azerbaijan',
    'Bosnia and Herzegovina', 'Chile',
    'Czech Republic', 'Germany', 'Spain',
    'Greece', 'Croatia', 'Italy',
    'Kyrgyzstan', 'Laos', 'Macedonia',
    'Mali', 'Nicaragua', 'Papua New Guinea', 'Romania', 'Somalia',
    'Serbia', 'Slovakia', 'Slovenia',
    'Chad', 'Thailand', 'Turkmenistan',
    'East Timor', 'Turkey', 'Venezuela',
    'Armenia', 'Bhutan', 'Central African Republic', 'Switzerland', 'Cuba',
    'Ecuador', 'Guyana', 'Cambodia',
    'Liberia', 'Myanmar', 'Namibia',
    'Suriname', 'Sweden', 'South Africa',
    'Zimbabwe'
]

countries_names = pd.read_csv('country_names.csv',encoding='latin-1').rename({'Country Code': 'ISO3'}, axis=1)
countries_names = countries_names.iloc[[e in all_countries for e in list(countries_names['name'].values)], :]

diet_2000 = pd.read_csv('Consumption_2000_FAOSTAT.csv')
LS_2000_df = diet_2000[diet_2000['Item']=='Animal Products'][['Country','Value']].merge(countries_names[['name','ISO3']].drop_duplicates(), right_on='name',left_on='Country',how='left')
# kcal/capita/day to cal/capita
LS_2000_df['LS2000_percapita'] = LS_2000_df['Value'].apply(lambda x:x*1e3*365.25)
LS_2000_df.head()

,Country,Value,name,ISO3,LS2000_percapita
0,Afghanistan,275,Afghanistan,AFG,100443750.0
1,Albania,674,Albania,ALB,246178500.0
2,Algeria,277,Algeria,DZA,101174250.0
3,Angola,153,Angola,AGO,55883250.0
4,Antigua and Barbuda,701,NaN,NaN,256040250.0


In [17]:
# function that returns the population from a dataframe by year
def get_population(X, year=2050):
    return int(X['population_'+str(year)].sum())

# function that returns the total production from a dataframe by year
def get_production(X, year=2050):
    return X['calories_'+str(year)].sum()

# function that calculates the caloric sufficiency. it is fed by a dataframe [to preserve generality]
def cal_suff_small(X, cntry=None, year=2050, perc_feed = .29, diet=.19, pop_fact=1., ADER=2320*1e3*365.25):
    
    food_waste=.19
    feed_food_factor=2.3
    
    conso = 1-food_waste
    perc_food = conso-perc_feed
    
    prod = get_production(X, year=year)
    food = perc_food * prod
    feed_now_food = 0.
    if cntry in LS_2000_df.ISO3.unique():
        LS = LS_2000_df[LS_2000_df.ISO3==cntry]['LS2000_percapita'].values * get_population(X, year=2000)
        
    else:
        LS = (1.1483e15*1e3)  * get_population(X, year=2000) / 5976296907
    if year==2050:
        LS_2000 = LS
        LS = diet * (food + feed_food_factor * LS_2000) / (1 + diet * (feed_food_factor - 1))
        feed_now_food = feed_food_factor * (LS_2000 - LS)
    
    production = food + LS + feed_now_food
    demand = get_population(X, year=year) * pop_fact * ADER
    
    return production/demand

In [18]:
ssp = 5
data_path = 'outputs/compare/'
df = pd.read_csv(data_path+'ssp'+str(ssp)+'_compare_new.csv')

cal_suff_small(df[df.ISO3=='USA'], cntry='USA', year=2050)

array([2.21996062])

In [28]:
def delta_cal_suff(x, country='MAR'):
#     x[0] = perc_diet : percentage of livestock in diet
#     x[1] = perc_feed : percentage of production that goes to feeding ls

    df_dup = df.copy()
    if country:
        df_dup = df[df['ISO3']==country].copy()
    
    cal_suff_2000 = cal_suff_small(df_dup, cntry=country, year=2000)
    cal_suff_2050 = cal_suff_small(df_dup, cntry=country, year=2050, diet=x[0], perc_feed=x[1])
    
    scnd_eq = x[0] - x[1] / (1 - .19) / 2.3
    
    return [cal_suff_2050 - cal_suff_2000, scnd_eq]


delta_cal_suff(x=[.19, .29])

[array([0.14563718]), 0.03433709071390234]

In [29]:
sol = root(delta_cal_suff, x0=[.19, .29], method='hybr').x

In [30]:
sol, delta_cal_suff(sol)[0]

(array([0.21920667, 0.40838203]), array([-1.11022302e-16]))